In [3]:
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from modules import simulator, geometry, coreConfig, dataHandler

def run_lvdt_simulation(path, filename, simulation_freq, simulation_amplitude, core_params,  coil_params):
    """
    Run the LVDT simulation with specified parameters and save the results to a file.

    Parameters:
        path (str): Path to save the data file.
        filename (str): Name of the data file.
        simulation_freq (float): Simulation frequency in Hz.
        simulation_amplitude (float): Amplitude for the outer coil circuit.
        core_params (tuple): Tuple containing (outer_diameter, inner_diameter) for the magnet geometry.
        coil_params (dict): Dictionary containing parameters for CoreCoil, MiddleCoil, and OuterCoil.
    """
    # Define geometry for core and coils
    Magnet_geo = geometry.def_core_geo(*core_params['MagnetCore'])
    MiddleCoil_geo = geometry.def_coil_geo(*coil_params['MiddleCoil'])
    OuterCoil_geo = geometry.def_coil_geo(*coil_params['OuterCoil'])

    # Define circuit properties
    MC_circuit = simulator.def_circuit_prop("middlecoil", 0, 0)
    OC_upper_circuit = simulator.def_circuit_prop("outercoil_upper", simulation_freq, simulation_amplitude)
    OC_lower_circuit = simulator.def_circuit_prop("outercoil_lower", simulation_freq, -simulation_amplitude)

    # Set up simulation environment
    simulator.def_femm_problem(signal_frequency=simulation_freq)
    simulator.build_air_geometry("Outside", 10)

    # Build geometry and assign labels
    m_label = simulator.build_core_geometry(Magnet_geo, 1)
    mc_label = simulator.build_coil_geometry(MiddleCoil_geo, MC_circuit, 2, customized_material=True)
    oc_upper_label = simulator.build_coil_geometry(OuterCoil_geo, OC_upper_circuit, 3, customized_material=True)
    oc_lower_label = simulator.build_coil_geometry(OuterCoil_geo, OC_lower_circuit, 4, customized_material=True, reverse=True)

    print("Magnet label:", m_label)
    print("Middle coil label:", mc_label)
    print("Outer coil upper label:", oc_upper_label)
    print("Outer coil lower label:", oc_lower_label)

    # Simulation configuration
    config = coreConfig.moving_config(-5, 1, 10)
    lvdt_data = simulator.def_lvdt_data(config['steps'])

    # Run the simulation
    sim_results = simulator.lvdt_simulation(
        moving_parts_label=[1],
        CC_config=config,
        lvdt_data=lvdt_data,
        OC_upper_circuit=OC_upper_circuit,
        OC_lower_circuit=OC_lower_circuit,
        MC_circuit=MC_circuit,
        CC_circuit=None
    )

    # Save results
    dataHandler.save_data(sim_results, path + filename)
    print("Data saved to:", path + filename)

In [5]:
filepath = "../data/magcore/"

simulation_freq = 10000
simulation_amplitude = 0.02
core_params = {
    'MangetCore': [4, 8, "N40"]
}   
coil_params = {
    'CoreCoil': ("100um", 0.1, 0, 8, 12, 12, 0),
    'MiddleCoil': ("100um", 0.1, 0, 16, 3, 18, 0),
    'OuterCoil': ("100um", 0.1, 0, 16, 3, 18, 16)
}

filename_template = "LVDT_magcore_d{diam:02d}_l{len:02d}.h5"
mag_diameter = [4,5,6,7,8]
mag_length   = [4,6,8,10,12]
mag_iter = coreConfig.params_iter(param1=mag_diameter, param2=mag_length)

for iter in mag_iter:
    core_params['MangetCore'] = [iter[0], iter[1], "N40"]
    filename = filename_template.format(diam=iter[0], len=iter[1])
    run_lvdt_simulation(filepath, filename, simulation_freq, simulation_amplitude, core_params, coil_params)

LVDT_magcore_d04_l04.h5
LVDT_magcore_d04_l06.h5
LVDT_magcore_d04_l08.h5
LVDT_magcore_d04_l10.h5
LVDT_magcore_d04_l12.h5
LVDT_magcore_d05_l04.h5
LVDT_magcore_d05_l06.h5
LVDT_magcore_d05_l08.h5
LVDT_magcore_d05_l10.h5
LVDT_magcore_d05_l12.h5
LVDT_magcore_d06_l04.h5
LVDT_magcore_d06_l06.h5
LVDT_magcore_d06_l08.h5
LVDT_magcore_d06_l10.h5
LVDT_magcore_d06_l12.h5
LVDT_magcore_d07_l04.h5
LVDT_magcore_d07_l06.h5
LVDT_magcore_d07_l08.h5
LVDT_magcore_d07_l10.h5
LVDT_magcore_d07_l12.h5
LVDT_magcore_d08_l04.h5
LVDT_magcore_d08_l06.h5
LVDT_magcore_d08_l08.h5
LVDT_magcore_d08_l10.h5
LVDT_magcore_d08_l12.h5
